<h1><center>Fig7 stats

This notebook contains statistical analysis of the data from the figure 7.

# Install libraries

Install all necessary libraries to run statistical tests in R.

In [ ]:
# Use R in Jupyter Notebook
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("readxl")
install.packages("tidyverse")
install.packages("emmeans")
install.packages("maptools", repos = "https://cloud.r-project.org")
install.packages("car", repos = "https://cloud.r-project.org")
install.packages("cowplot", repos = "https://cloud.r-project.org")
install.packages("rstatix", repos = "https://cloud.r-project.org")
install.packages("ggpubr", repos = "https://cloud.r-project.org")

In [ ]:
%%R
library("readxl")
library("tidyverse")
library("emmeans")
library("maptools")
library("car")
library("cowplot")
library("rstatix")
library("ggpubr")

# Directory

Set directory with the source data. Mount disk if you use Google Colab.

In [ ]:
# Mount disk if you use Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change directory
%cd /content/drive/My\ Drive/!lab/Manuscripts/LHoutputs/full submission 2/sourcedata

/content/drive/My Drive/!lab/Manuscripts/LHoutputs/full submission 2/sourcedata


# OFT total mobility score Fig7C,E


In [ ]:
%%R
df <- read_excel("fig7.xlsx", sheet = "OFTtotal")
df$group = as.factor(df$group)
df$output = as.factor(df$output)
df$period = as.factor(df$period)
df$laser = as.factor(df$laser)
df$value = as.numeric(df$value)

Outliers

In [ ]:
%%R
df %>%
  group_by(group,test,output,laser) %>%
  identify_outliers(value)

# A tibble: 17 x 9
   laser group test  output mouse   value period  is.outlier is.extreme
   <fct> <fct> <chr> <fct>  <chr>   <dbl> <fct>   <lgl>      <lgl>     
 1 OFF   ChR2  OFT   DRN    LHA99  0.204  period2 TRUE       FALSE     
 2 OFF   ChR2  OFT   DRN    LHA99  0.226  period3 TRUE       FALSE     
 3 OFF   ChR2  OFT   DRN    LHA99  0.207  period4 TRUE       FALSE     
 4 OFF   ChR2  OFT   DRN    LHA99  0.213  period5 TRUE       FALSE     
 5 OFF   ChR2  OFT   LHb    LHA99  0.254  period1 TRUE       FALSE     
 6 ON    ChR2  OFT   LHb    LHA98  0.0777 period1 TRUE       FALSE     
 7 OFF   ChR2  OFT   VTA    LHA101 0.264  period1 TRUE       FALSE     
 8 OFF   eYFP  OFT   DRN    LHA61  0.285  period1 TRUE       FALSE     
 9 OFF   eYFP  OFT   LHb    LHA40  0.0199 period1 TRUE       FALSE     
10 OFF   eYFP  OFT   LHb    LHA61  0.242  period1 TRUE       FALSE     
11 OFF   eYFP  OFT   LHb    LHA61  0.251  period2 TRUE       FALSE     
12 OFF   eYFP  OFT   LHb    LHA61  0.261  per

No extreme outliers

Normality

In [ ]:
%%R
shapiro <- df %>%
  group_by(test) %>%
  shapiro_test(value)
print(shapiro,n=24)

ANOVA

In [ ]:
%%R
res.aov <- anova_test(data=df, dv=value, wid=mouse, between=group, within=c(output,period,laser))
get_anova_table(res.aov)

ANOVA Table (type III tests)

                      Effect  DFn    DFd      F        p p<.05   ges
1                      group 1.00  14.00  0.300 0.593000       0.007
2                     output 2.00  28.00  1.358 0.274000       0.016
3                     period 1.36  19.02  0.752 0.436000       0.006
4                      laser 1.00  14.00 27.405 0.000126     * 0.115
5               group:output 2.00  28.00  1.546 0.231000       0.019
6               group:period 1.36  19.02  0.998 0.357000       0.008
7                group:laser 1.00  14.00 15.749 0.001000     * 0.069
8              output:period 3.66  51.30  0.558 0.679000       0.004
9               output:laser 2.00  28.00  3.078 0.062000       0.013
10              period:laser 4.00  56.00  0.609 0.658000       0.002
11       group:output:period 3.66  51.30  1.581 0.197000       0.013
12        group:output:laser 2.00  28.00  1.743 0.193000       0.008
13        group:period:laser 4.00  56.00  2.984 0.026000     * 0.009
14  

In [ ]:
%%R
two.way <- df %>%
  group_by(group,output) %>%
  anova_test(dv = value, wid = mouse, within = c(period,laser))
get_anova_table(two.way)

# A tibble: 18 x 9
   group output Effect         DFn   DFd      F        p `p<.05`   ges
   <fct> <fct>  <chr>        <dbl> <dbl>  <dbl>    <dbl> <chr>   <dbl>
 1 ChR2  DRN    period        4    32     0.767 0.555    ""      0.012
 2 ChR2  DRN    laser         1     8    10.5   0.012    "*"     0.176
 3 ChR2  DRN    period:laser  4    32     2.08  0.106    ""      0.035
 4 ChR2  LHb    period        1.43 11.5   1.27  0.304    ""      0.04 
 5 ChR2  LHb    laser         1     8    44.9   0.000153 "*"     0.479
 6 ChR2  LHb    period:laser  4    32     0.15  0.962    ""      0.004
 7 ChR2  VTA    period        1.38  9.68  0.347 0.639    ""      0.012
 8 ChR2  VTA    laser         1     7    10.4   0.014    "*"     0.313
 9 ChR2  VTA    period:laser  4    28     0.592 0.671    ""      0.007
10 eYFP  DRN    period        1.59 11.1   0.484 0.587    ""      0.021
11 eYFP  DRN    laser         1     7     0.159 0.702    ""      0.001
12 eYFP  DRN    period:laser  1.73 12.1   1.09  0.357    "

In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ laser, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
pwc

# A tibble: 6 x 8
  group output group1 group2 conf.low conf.high    p.adj p.adj.signif
  <fct> <fct>  <chr>  <chr>     <dbl>     <dbl>    <dbl> <chr>       
1 ChR2  DRN    OFF    ON      0.0193     0.0534 5.66e- 5 ****        
2 ChR2  LHb    OFF    ON      0.0605     0.0959 3.56e-10 ****        
3 ChR2  VTA    OFF    ON      0.0404     0.0815 8.53e- 8 ****        
4 eYFP  DRN    OFF    ON     -0.0191     0.0259 7.64e- 1 ns          
5 eYFP  LHb    OFF    ON     -0.0135     0.0313 4.34e- 1 ns          
6 eYFP  VTA    OFF    ON     -0.00942    0.0318 2.83e- 1 ns          


In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ period, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
print(pwc,n=60)

# A tibble: 60 x 8
   group output group1  group2  conf.low conf.high p.adj p.adj.signif
   <fct> <fct>  <chr>   <chr>      <dbl>     <dbl> <dbl> <chr>       
 1 ChR2  DRN    period1 period2  -0.0374    0.0466 0.998 ns          
 2 ChR2  DRN    period1 period3  -0.0308    0.0532 0.946 ns          
 3 ChR2  DRN    period1 period4  -0.0397    0.0443 1     ns          
 4 ChR2  DRN    period1 period5  -0.0428    0.0412 1     ns          
 5 ChR2  DRN    period2 period3  -0.0354    0.0486 0.992 ns          
 6 ChR2  DRN    period2 period4  -0.0443    0.0397 1     ns          
 7 ChR2  DRN    period2 period5  -0.0473    0.0367 0.997 ns          
 8 ChR2  DRN    period3 period4  -0.0509    0.0331 0.976 ns          
 9 ChR2  DRN    period3 period5  -0.0539    0.0301 0.932 ns          
10 ChR2  DRN    period4 period5  -0.0450    0.0390 1     ns          
11 ChR2  LHb    period1 period2  -0.0741    0.0341 0.841 ns          
12 ChR2  LHb    period1 period3  -0.0691    0.0391 0.938 ns          
1

# OFT mobility score when mobile FigD,F (left)


In [ ]:
%%R
df <- read_excel("fig7.xlsx", sheet = "OFTmovement")
df$group = as.factor(df$group)
df$output = as.factor(df$output)
df$period = as.factor(df$period)
df$laser = as.factor(df$laser)
df$value = as.numeric(df$value)
colnames(df) <- c('mouse','value','period','laser','group','test','output')

Outliers

In [ ]:
%%R
df %>%
  group_by(group,output,period,laser) %>%
  identify_outliers(value)

# A tibble: 31 x 9
   period  laser group output mouse value test  is.outlier is.extreme
   <fct>   <fct> <fct> <fct>  <chr> <dbl> <chr> <lgl>      <lgl>     
 1 period1 ON    ChR2  DRN    LHA99 0.250 OFT   TRUE       FALSE     
 2 period2 ON    ChR2  DRN    LHA93 0.138 OFT   TRUE       FALSE     
 3 period3 OFF   ChR2  DRN    LHA93 0.250 OFT   TRUE       FALSE     
 4 period3 OFF   ChR2  DRN    LHA99 0.244 OFT   TRUE       FALSE     
 5 period3 ON    ChR2  DRN    LHA93 0.120 OFT   TRUE       TRUE      
 6 period4 ON    ChR2  DRN    LHA99 0.228 OFT   TRUE       FALSE     
 7 period5 OFF   ChR2  DRN    LHA95 0.128 OFT   TRUE       TRUE      
 8 period5 OFF   ChR2  DRN    LHA99 0.232 OFT   TRUE       TRUE      
 9 period5 ON    ChR2  DRN    LHA93 0.109 OFT   TRUE       FALSE     
10 period5 ON    ChR2  DRN    LHA99 0.218 OFT   TRUE       FALSE     
# … with 21 more rows


Normality

In [ ]:
%%R
shapiro <- df %>%
  group_by(group,output,period,laser) %>%
  shapiro_test(value)
print(shapiro,n=24)

ANOVA

In [ ]:
%%R
res.aov <- anova_test(data=df, dv=value, wid=mouse, between=group, within=c(output,period,laser))
get_anova_table(res.aov)

ANOVA Table (type III tests)

                      Effect  DFn    DFd     F     p p<.05   ges
1                      group 1.00  13.00 1.612 0.227       0.021
2                     output 2.00  26.00 1.159 0.330       0.013
3                     period 1.33  17.24 3.691 0.062       0.051
4                      laser 1.00  13.00 3.376 0.089       0.019
5               group:output 2.00  26.00 0.613 0.550       0.007
6               group:period 1.33  17.24 0.805 0.416       0.011
7                group:laser 1.00  13.00 0.622 0.445       0.004
8              output:period 3.48  45.29 0.839 0.494       0.009
9               output:laser 2.00  26.00 3.676 0.039     * 0.029
10              period:laser 4.00  52.00 0.351 0.842       0.001
11       group:output:period 3.48  45.29 1.281 0.292       0.013
12        group:output:laser 2.00  26.00 1.108 0.345       0.009
13        group:period:laser 4.00  52.00 1.832 0.137       0.007
14       output:period:laser 8.00 104.00 1.049 0.404       0

In [ ]:
%%R
two.way <- df %>%
  group_by(group,output) %>%
  anova_test(dv = value, wid = mouse, within = c(period,laser))
get_anova_table(two.way)

# A tibble: 18 x 9
   group output Effect         DFn   DFd      F     p `p<.05`      ges
   <fct> <fct>  <chr>        <dbl> <dbl>  <dbl> <dbl> <chr>      <dbl>
 1 ChR2  DRN    period        4    32     4.74  0.004 "*"     0.106   
 2 ChR2  DRN    laser         1     8     0.34  0.576 ""      0.012   
 3 ChR2  DRN    period:laser  4    32     1.85  0.144 ""      0.048   
 4 ChR2  LHb    period        1.26 10.0   1.62  0.239 ""      0.088   
 5 ChR2  LHb    laser         1     8    12.0   0.009 "*"     0.248   
 6 ChR2  LHb    period:laser  4    32     0.922 0.463 ""      0.019   
 7 ChR2  VTA    period        4    24     6.36  0.001 "*"     0.118   
 8 ChR2  VTA    laser         1     6     0.671 0.444 ""      0.04    
 9 ChR2  VTA    period:laser  4    24     0.948 0.454 ""      0.03    
10 eYFP  DRN    period        1.5  10.5   2.31  0.154 ""      0.105   
11 eYFP  DRN    laser         1     7     0.017 0.901 ""      0.000313
12 eYFP  DRN    period:laser  4    28     0.937 0.457 ""  

In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ laser, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
print(pwc,n=30)

# A tibble: 6 x 8
  group output group1 group2 conf.low conf.high     p.adj p.adj.signif
  <fct> <fct>  <chr>  <chr>     <dbl>     <dbl>     <dbl> <chr>       
1 ChR2  DRN    OFF    ON     -0.0193    0.00676 0.342     ns          
2 ChR2  LHb    OFF    ON      0.0238    0.0541  0.0000019 ****        
3 ChR2  VTA    OFF    ON     -0.00530   0.0283  0.177     ns          
4 eYFP  DRN    OFF    ON     -0.0165    0.0142  0.884     ns          
5 eYFP  LHb    OFF    ON     -0.00456   0.0279  0.157     ns          
6 eYFP  VTA    OFF    ON     -0.00977   0.0204  0.485     ns          


In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ period, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
print(pwc,n=60)

# A tibble: 60 x 8
   group output group1  group2  conf.low conf.high  p.adj p.adj.signif
   <fct> <fct>  <chr>   <chr>      <dbl>     <dbl>  <dbl> <chr>       
 1 ChR2  DRN    period1 period2  -0.0366  0.0194   0.911  ns          
 2 ChR2  DRN    period1 period3  -0.0425  0.0135   0.601  ns          
 3 ChR2  DRN    period1 period4  -0.0486  0.00733  0.248  ns          
 4 ChR2  DRN    period1 period5  -0.0565 -0.000556 0.0433 *           
 5 ChR2  DRN    period2 period3  -0.0339  0.0221   0.977  ns          
 6 ChR2  DRN    period2 period4  -0.0400  0.0160   0.753  ns          
 7 ChR2  DRN    period2 period5  -0.0479  0.00807  0.283  ns          
 8 ChR2  DRN    period3 period4  -0.0341  0.0218   0.973  ns          
 9 ChR2  DRN    period3 period5  -0.0420  0.0139   0.631  ns          
10 ChR2  DRN    period4 period5  -0.0359  0.0201   0.934  ns          
11 ChR2  LHb    period1 period2  -0.0560  0.0192   0.653  ns          
12 ChR2  LHb    period1 period3  -0.0609  0.0143   0.424  

# OFT duration of bouts FigD,F (middle)


In [ ]:
%%R
df <- read_excel("fig7.xlsx", sheet = "OFTduration")
df$group = as.factor(df$group)
df$output = as.factor(df$output)
df$period = as.factor(df$period)
df$laser = as.factor(df$laser)
df$value = as.numeric(df$value)

Outliers

In [ ]:
%%R
df %>%
  group_by(group,test,output,laser) %>%
  identify_outliers(value)

# A tibble: 35 x 9
   laser group test  output mouse  value period  is.outlier is.extreme
   <fct> <fct> <chr> <fct>  <chr>  <dbl> <fct>   <lgl>      <lgl>     
 1 OFF   ChR2  OFT   DRN    LHA99   21.8 period2 TRUE       TRUE      
 2 OFF   ChR2  OFT   DRN    LHA99   22.0 period3 TRUE       TRUE      
 3 ON    ChR2  OFT   DRN    LHA99   23.2 period1 TRUE       FALSE     
 4 ON    ChR2  OFT   DRN    LHA101  22.3 period2 TRUE       FALSE     
 5 ON    ChR2  OFT   DRN    LHA101  22.3 period3 TRUE       FALSE     
 6 ON    ChR2  OFT   DRN    LHA99   29.5 period3 TRUE       TRUE      
 7 ON    ChR2  OFT   DRN    LHA99   22.7 period4 TRUE       FALSE     
 8 OFF   ChR2  OFT   LHb    LHA101  15.2 period1 TRUE       FALSE     
 9 OFF   ChR2  OFT   LHb    LHA99   18.1 period1 TRUE       TRUE      
10 OFF   ChR2  OFT   LHb    LHA101  12.0 period2 TRUE       FALSE     
# … with 25 more rows


Normality

In [ ]:
%%R
shapiro <- df %>%
  group_by(group,output,laser) %>%
  shapiro_test(value)
print(shapiro,n=24)

ANOVA

In [ ]:
%%R
res.aov <- anova_test(data=df, dv=value, wid=mouse, between=group, within=c(output,period,laser))
get_anova_table(res.aov)

ANOVA Table (type III tests)

                      Effect  DFn    DFd     F     p p<.05      ges
1                      group 1.00  13.00 0.139 0.715       0.003000
2                     output 1.39  18.09 0.449 0.575       0.006000
3                     period 4.00  52.00 2.788 0.036     * 0.016000
4                      laser 1.00  13.00 9.431 0.009     * 0.036000
5               group:output 1.39  18.09 3.340 0.073       0.044000
6               group:period 4.00  52.00 0.911 0.465       0.005000
7                group:laser 1.00  13.00 7.914 0.015     * 0.030000
8              output:period 3.46  45.03 0.660 0.602       0.007000
9               output:laser 2.00  26.00 3.535 0.044     * 0.015000
10              period:laser 4.00  52.00 0.057 0.994       0.000332
11       group:output:period 3.46  45.03 1.568 0.205       0.017000
12        group:output:laser 2.00  26.00 2.866 0.075       0.012000
13        group:period:laser 4.00  52.00 0.387 0.817       0.002000
14       output:pe

In [ ]:
%%R
two.way <- df %>%
  group_by(group,output) %>%
  anova_test(dv = value, wid = mouse, within = c(period,laser))
get_anova_table(two.way)

# A tibble: 18 x 9
   group output Effect         DFn   DFd      F     p `p<.05`      ges
   <fct> <fct>  <chr>        <dbl> <dbl>  <dbl> <dbl> <chr>      <dbl>
 1 ChR2  DRN    period        4    32     1.42  0.249 ""      0.026   
 2 ChR2  DRN    laser         1     8     4.16  0.076 ""      0.054   
 3 ChR2  DRN    period:laser  1.88 15.1   0.056 0.938 ""      0.001   
 4 ChR2  LHb    period        1.57 12.6   2.00  0.18  ""      0.063   
 5 ChR2  LHb    laser         1     8     2.99  0.122 ""      0.047   
 6 ChR2  LHb    period:laser  1.21  9.69  0.183 0.725 ""      0.008   
 7 ChR2  VTA    period        4    24     0.977 0.438 ""      0.032   
 8 ChR2  VTA    laser         1     6    10.7   0.017 "*"     0.297   
 9 ChR2  VTA    period:laser  4    24     1.78  0.165 ""      0.025   
10 eYFP  DRN    period        4    28     0.928 0.462 ""      0.046   
11 eYFP  DRN    laser         1     7     0.154 0.707 ""      0.000559
12 eYFP  DRN    period:laser  4    28     0.829 0.518 ""  

In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ laser, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
pwc

# A tibble: 6 x 8
  group output group1 group2 conf.low conf.high     p.adj p.adj.signif
  <fct> <fct>  <chr>  <chr>     <dbl>     <dbl>     <dbl> <chr>       
1 ChR2  DRN    OFF    ON       0.260       4.69 0.029     *           
2 ChR2  LHb    OFF    ON       0.0432      5.54 0.0466    *           
3 ChR2  VTA    OFF    ON       3.89       10.2  0.0000273 ****        
4 eYFP  DRN    OFF    ON      -2.20        1.79 0.841     ns          
5 eYFP  LHb    OFF    ON      -3.10        3.96 0.809     ns          
6 eYFP  VTA    OFF    ON      -1.23        1.87 0.684     ns          


In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ period, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
print(pwc,n=60)

# A tibble: 60 x 8
   group output group1  group2  conf.low conf.high p.adj p.adj.signif
   <fct> <fct>  <chr>   <chr>      <dbl>     <dbl> <dbl> <chr>       
 1 ChR2  DRN    period1 period2    -6.16      3.99 0.976 ns          
 2 ChR2  DRN    period1 period3    -3.87      6.28 0.964 ns          
 3 ChR2  DRN    period1 period4    -6.09      4.06 0.981 ns          
 4 ChR2  DRN    period1 period5    -5.64      4.51 0.998 ns          
 5 ChR2  DRN    period2 period3    -2.79      7.36 0.72  ns          
 6 ChR2  DRN    period2 period4    -5.01      5.14 1     ns          
 7 ChR2  DRN    period2 period5    -4.56      5.59 0.999 ns          
 8 ChR2  DRN    period3 period4    -7.29      2.86 0.741 ns          
 9 ChR2  DRN    period3 period5    -6.84      3.31 0.868 ns          
10 ChR2  DRN    period4 period5    -4.63      5.53 0.999 ns          
11 ChR2  LHb    period1 period2    -9.22      3.08 0.635 ns          
12 ChR2  LHb    period1 period3   -10.5       1.82 0.293 ns          
1

# OFT number of bouts FigD,F (left)


In [ ]:
%%R
df <- read_excel("fig7.xlsx", sheet = "OFTnumber")
df$group = as.factor(df$group)
df$output = as.factor(df$output)
df$period = as.factor(df$period)
df$laser = as.factor(df$laser)
df$value = as.numeric(df$value)

Outliers

In [ ]:
%%R
df %>%
  group_by(group,test,output,laser) %>%
  identify_outliers(value)

# A tibble: 2 x 9
  laser group test  output mouse  value period  is.outlier is.extreme
  <fct> <fct> <chr> <fct>  <chr>  <dbl> <fct>   <lgl>      <lgl>     
1 OFF   ChR2  OFT   DRN    LHA98      2 period1 TRUE       FALSE     
2 OFF   ChR2  OFT   DRN    LHA100    22 period4 TRUE       TRUE      


Normality

In [ ]:
%%R
shapiro <- df %>%
  group_by(group,output,laser) %>%
  shapiro_test(value)
shapiro

ANOVA

In [ ]:
%%R
res.aov <- anova_test(data=df, dv=value, wid=mouse, between=group, within=c(output,period,laser))
get_anova_table(res.aov)

ANOVA Table (type III tests)

                      Effect DFn DFd      F        p p<.05      ges
1                      group   1  14  2.181 1.62e-01       0.049000
2                     output   2  28  0.090 9.15e-01       0.000801
3                     period   4  56 19.038 6.21e-10     * 0.090000
4                      laser   1  14  0.876 3.65e-01       0.006000
5               group:output   2  28  1.377 2.69e-01       0.012000
6               group:period   4  56  1.287 2.86e-01       0.007000
7                group:laser   1  14  1.578 2.30e-01       0.011000
8              output:period   8 112  1.890 6.80e-02       0.021000
9               output:laser   2  28  7.694 2.00e-03     * 0.017000
10              period:laser   4  56  3.732 9.00e-03     * 0.020000
11       group:output:period   8 112  0.581 7.92e-01       0.007000
12        group:output:laser   2  28 10.422 4.14e-04     * 0.024000
13        group:period:laser   4  56  1.171 3.33e-01       0.006000
14       output:pe

In [ ]:
%%R
two.way <- df %>%
  group_by(group,output) %>%
  anova_test(dv = value, wid = mouse, within = c(period,laser))
get_anova_table(two.way)

# A tibble: 18 x 9
   group output Effect         DFn   DFd      F        p `p<.05`        ges
   <fct> <fct>  <chr>        <dbl> <dbl>  <dbl>    <dbl> <chr>        <dbl>
 1 ChR2  DRN    period         4    32    1.06  0.393    ""      0.033     
 2 ChR2  DRN    laser          1     8    3.45  0.1      ""      0.105     
 3 ChR2  DRN    period:laser   1.7  13.6  1.52  0.251    ""      0.042     
 4 ChR2  LHb    period         4    32    1.32  0.285    ""      0.042     
 5 ChR2  LHb    laser          1     8   12.8   0.007    "*"     0.26      
 6 ChR2  LHb    period:laser   4    32    1.51  0.222    ""      0.032     
 7 ChR2  VTA    period         4    28    6.40  0.000867 "*"     0.206     
 8 ChR2  VTA    laser          1     7    0.977 0.356    ""      0.045     
 9 ChR2  VTA    period:laser   4    28    4.86  0.004    "*"     0.092     
10 eYFP  DRN    period         4    28    2.74  0.049    "*"     0.11      
11 eYFP  DRN    laser          1     7    0.005 0.944    ""      0.00

In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ laser, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
pwc

# A tibble: 6 x 8
  group output group1 group2 conf.low conf.high       p.adj p.adj.signif
  <fct> <fct>  <chr>  <chr>     <dbl>     <dbl>       <dbl> <chr>       
1 ChR2  DRN    OFF    ON        0.930     4.27  0.00265     **          
2 ChR2  LHb    OFF    ON        2.99      6.52  0.000000672 ****        
3 ChR2  VTA    OFF    ON       -3.42      0.318 0.103       ns          
4 eYFP  DRN    OFF    ON       -2.09      2.14  0.981       ns          
5 eYFP  LHb    OFF    ON       -2.96      1.76  0.613       ns          
6 eYFP  VTA    OFF    ON       -2.21      2.01  0.925       ns          


In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ period, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
print(pwc,n=60)

# A tibble: 60 x 8
   group output group1  group2  conf.low conf.high   p.adj p.adj.signif
   <fct> <fct>  <chr>   <chr>      <dbl>     <dbl>   <dbl> <chr>       
 1 ChR2  DRN    period1 period2   -2.58       5.24 0.876   ns          
 2 ChR2  DRN    period1 period3   -2.97       4.86 0.962   ns          
 3 ChR2  DRN    period1 period4   -1.80       6.02 0.563   ns          
 4 ChR2  DRN    period1 period5   -3.24       4.58 0.989   ns          
 5 ChR2  DRN    period2 period3   -4.30       3.52 0.999   ns          
 6 ChR2  DRN    period2 period4   -3.13       4.69 0.981   ns          
 7 ChR2  DRN    period2 period5   -4.58       3.24 0.989   ns          
 8 ChR2  DRN    period3 period4   -2.74       5.08 0.92    ns          
 9 ChR2  DRN    period3 period5   -4.19       3.63 1       ns          
10 ChR2  DRN    period4 period5   -5.36       2.47 0.841   ns          
11 ChR2  LHb    period1 period2   -3.35       5.68 0.951   ns          
12 ChR2  LHb    period1 period3   -1.90      

# TST total mobility score Fig7H,J


In [ ]:
%%R
df <- read_excel("fig7.xlsx", sheet = "TSTtotal")
df$group = as.factor(df$group)
df$output = as.factor(df$output)
df$period = as.factor(df$period)
df$laser = as.factor(df$laser)
df$value = as.numeric(df$value)

Outliers

In [ ]:
%%R
df %>%
  group_by(group,test,output,laser) %>%
  identify_outliers(value)

# A tibble: 5 x 9
  laser group test  output mouse  value period  is.outlier is.extreme
  <fct> <fct> <chr> <fct>  <chr>  <dbl> <fct>   <lgl>      <lgl>     
1 OFF   ChR2  TST   DRN    LHA95  0.221 period1 TRUE       FALSE     
2 ON    ChR2  TST   VTA    LHA98  0.320 period1 TRUE       FALSE     
3 ON    ChR2  TST   VTA    LHA94  0.299 period3 TRUE       FALSE     
4 ON    eYFP  TST   DRN    LHA106 0.219 period1 TRUE       FALSE     
5 ON    eYFP  TST   VTA    LHA38  0.220 period1 TRUE       FALSE     


No extreme outliers

Normality

In [ ]:
%%R
shapiro <- df %>%
  group_by(group,output,laser) %>%
  shapiro_test(value)
print(shapiro,n=24)

ANOVA

In [ ]:
%%R
res.aov <- anova_test(data=df, dv=value, wid=mouse, between=group, within=c(output,period,laser))
get_anova_table(res.aov)

ANOVA Table (type III tests)

                      Effect  DFn    DFd      F        p p<.05   ges
1                      group 1.00  15.00 16.990 9.05e-04     * 0.148
2                     output 2.00  30.00  1.181 3.21e-01       0.013
3                     period 4.00  60.00 61.035 1.79e-20     * 0.230
4                      laser 1.00  15.00  6.771 2.00e-02     * 0.015
5               group:output 2.00  30.00  0.496 6.14e-01       0.005
6               group:period 4.00  60.00  2.077 9.50e-02       0.010
7                group:laser 1.00  15.00 22.032 2.88e-04     * 0.048
8              output:period 3.75  56.32  0.987 4.18e-01       0.015
9               output:laser 2.00  30.00  0.261 7.72e-01       0.002
10              period:laser 4.00  60.00  3.038 2.40e-02     * 0.016
11       group:output:period 3.75  56.32  1.150 3.42e-01       0.017
12        group:output:laser 2.00  30.00  1.004 3.78e-01       0.009
13        group:period:laser 4.00  60.00  0.448 7.74e-01       0.002
14  

In [ ]:
%%R
two.way <- df %>%
  group_by(group,output) %>%
  anova_test(dv = value, wid = mouse, within = c(period,laser))
get_anova_table(two.way)

# A tibble: 18 x 9
   group output Effect         DFn   DFd         F          p `p<.05`        ges
   <fct> <fct>  <chr>        <dbl> <dbl>     <dbl>      <dbl> <chr>        <dbl>
 1 ChR2  DRN    period        4     32    5.08     0.003      "*"     0.127     
 2 ChR2  DRN    laser         1      8    2.46     0.155      ""      0.085     
 3 ChR2  DRN    period:laser  4     32    2.27     0.084      ""      0.059     
 4 ChR2  LHb    period        2.32  18.5  9.62     0.000946   "*"     0.204     
 5 ChR2  LHb    laser         1      8    0.927    0.364      ""      0.039     
 6 ChR2  LHb    period:laser  4     32    0.908    0.471      ""      0.021     
 7 ChR2  VTA    period        4     32   12.8      0.00000245 "*"     0.279     
 8 ChR2  VTA    laser         1      8   12.0      0.009      "*"     0.27      
 9 ChR2  VTA    period:laser  4     32    1.46     0.239      ""      0.047     
10 eYFP  DRN    period        1.72  13.8  8.53     0.005      "*"     0.383     
11 eYFP  

In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ laser, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
pwc

# A tibble: 6 x 8
  group output group1 group2 conf.low conf.high      p.adj p.adj.signif
  <fct> <fct>  <chr>  <chr>     <dbl>     <dbl>      <dbl> <chr>       
1 ChR2  DRN    OFF    ON      0.00694    0.0521 0.011      *           
2 ChR2  LHb    OFF    ON     -0.00367    0.0419 0.0991     ns          
3 ChR2  VTA    OFF    ON      0.0287     0.0698 0.00000752 ****        
4 eYFP  DRN    OFF    ON     -0.0359     0.0104 0.277      ns          
5 eYFP  LHb    OFF    ON     -0.0257     0.0260 0.992      ns          
6 eYFP  VTA    OFF    ON     -0.0376     0.0120 0.308      ns          


In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ period, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
print(pwc,n=60)

# TST mobility score when mobile FigI,K (left)


In [ ]:
%%R
df <- read_excel("fig7.xlsx", sheet = "TSTmovement")
df$group = as.factor(df$group)
df$output = as.factor(df$output)
df$period = as.factor(df$period)
df$laser = as.factor(df$laser)
df$value = as.numeric(df$value)

Outliers

In [ ]:
%%R
df %>%
  group_by(group,output,period,laser) %>%
  identify_outliers(value)

# A tibble: 22 x 9
   period  laser group output mouse  value test  is.outlier is.extreme
   <fct>   <fct> <fct> <fct>  <chr>  <dbl> <chr> <lgl>      <lgl>     
 1 period1 ON    ChR2  DRN    LHA101 0.118 TST   TRUE       FALSE     
 2 period1 ON    ChR2  DRN    LHA98  0.207 TST   TRUE       FALSE     
 3 period3 OFF   ChR2  DRN    LHA96  0.260 TST   TRUE       FALSE     
 4 period5 OFF   ChR2  DRN    LHA101 0.257 TST   TRUE       FALSE     
 5 period2 OFF   ChR2  LHb    LHA93  0.240 TST   TRUE       FALSE     
 6 period2 ON    ChR2  LHb    LHA98  0.103 TST   TRUE       FALSE     
 7 period1 ON    ChR2  VTA    LHA98  0.324 TST   TRUE       TRUE      
 8 period2 ON    ChR2  VTA    LHA96  0.113 TST   TRUE       FALSE     
 9 period3 ON    ChR2  VTA    LHA94  0.307 TST   TRUE       TRUE      
10 period4 OFF   ChR2  VTA    LHA94  0.272 TST   TRUE       TRUE      
# … with 12 more rows


Normality

In [ ]:
%%R
shapiro <- df %>%
  group_by(group,output,period,laser) %>%
  shapiro_test(value)
print(shapiro,n=24)

ANOVA

In [ ]:
%%R
res.aov <- anova_test(data=df, dv=value, wid=mouse, between=group, within=c(output,period,laser))
get_anova_table(res.aov)

ANOVA Table (type III tests)

                      Effect DFn DFd      F        p p<.05      ges
1                      group   1  10  3.981 7.40e-02       0.041000
2                     output   2  20  1.570 2.33e-01       0.028000
3                     period   4  40 28.115 3.76e-11     * 0.205000
4                      laser   1  10  0.037 8.51e-01       0.000045
5               group:output   2  20  0.023 9.78e-01       0.000413
6               group:period   4  40  3.537 1.50e-02     * 0.031000
7                group:laser   1  10  0.483 5.03e-01       0.000588
8              output:period   8  80  1.010 4.35e-01       0.037000
9               output:laser   2  20  0.610 5.53e-01       0.004000
10              period:laser   4  40  0.699 5.97e-01       0.003000
11       group:output:period   8  80  0.499 8.53e-01       0.019000
12        group:output:laser   2  20  1.564 2.34e-01       0.010000
13        group:period:laser   4  40  1.181 3.34e-01       0.006000
14       output:pe

In [ ]:
%%R
two.way <- df %>%
  group_by(group,output) %>%
  anova_test(dv = value, wid = mouse, within = c(period,laser))
get_anova_table(two.way)

# A tibble: 18 x 9
   group output Effect         DFn   DFd     F        p `p<.05`   ges
   <fct> <fct>  <chr>        <dbl> <dbl> <dbl>    <dbl> <chr>   <dbl>
 1 ChR2  DRN    period        4     20   2.97  0.045    "*"     0.118
 2 ChR2  DRN    laser         1      5   0.021 0.89     ""      0.001
 3 ChR2  DRN    period:laser  4     20   1.39  0.274    ""      0.085
 4 ChR2  LHb    period        4     24   4.80  0.005    "*"     0.209
 5 ChR2  LHb    laser         1      6   1.33  0.293    ""      0.048
 6 ChR2  LHb    period:laser  4     24   1.03  0.413    ""      0.042
 7 ChR2  VTA    period        1.9   15.2 6.85  0.008    "*"     0.236
 8 ChR2  VTA    laser         1      8   2.03  0.192    ""      0.032
 9 ChR2  VTA    period:laser  4     32   0.452 0.77     ""      0.014
10 eYFP  DRN    period        1.75  14.0 6.94  0.01     "*"     0.31 
11 eYFP  DRN    laser         1      8   2.54  0.149    ""      0.006
12 eYFP  DRN    period:laser  4     32   0.573 0.684    ""      0.013
1

In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ laser, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
print(pwc,n=30)

# A tibble: 6 x 8
  group output group1 group2 conf.low conf.high p.adj p.adj.signif
  <fct> <fct>  <chr>  <chr>     <dbl>     <dbl> <dbl> <chr>       
1 ChR2  DRN    OFF    ON     -0.0266     0.0116 0.438 ns          
2 ChR2  LHb    OFF    ON     -0.0271     0.0134 0.503 ns          
3 ChR2  VTA    OFF    ON     -0.00478    0.0323 0.144 ns          
4 eYFP  DRN    OFF    ON     -0.0326     0.0174 0.547 ns          
5 eYFP  LHb    OFF    ON     -0.0246     0.0360 0.709 ns          
6 eYFP  VTA    OFF    ON     -0.0399     0.0227 0.587 ns          


In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ period, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
print(pwc,n=60)

# A tibble: 60 x 8
   group output group1  group2  conf.low conf.high      p.adj p.adj.signif
   <fct> <fct>  <chr>   <chr>      <dbl>     <dbl>      <dbl> <chr>       
 1 ChR2  DRN    period1 period2  -0.0345   0.0497  0.987      ns          
 2 ChR2  DRN    period1 period3  -0.0535   0.0283  0.91       ns          
 3 ChR2  DRN    period1 period4  -0.0614   0.0229  0.707      ns          
 4 ChR2  DRN    period1 period5  -0.0638   0.0179  0.523      ns          
 5 ChR2  DRN    period2 period3  -0.0623   0.0219  0.668      ns          
 6 ChR2  DRN    period2 period4  -0.0702   0.0165  0.422      ns          
 7 ChR2  DRN    period2 period5  -0.0727   0.0116  0.265      ns          
 8 ChR2  DRN    period3 period4  -0.0488   0.0355  0.992      ns          
 9 ChR2  DRN    period3 period5  -0.0512   0.0305  0.955      ns          
10 ChR2  DRN    period4 period5  -0.0458   0.0384  0.999      ns          
11 ChR2  LHb    period1 period2  -0.0648   0.0199  0.579      ns          
12 ChR

# TST duration of bouts FigI,K (middle)


In [ ]:
%%R
df <- read_excel("fig7.xlsx", sheet = "TSTduration")
df$group = as.factor(df$group)
df$output = as.factor(df$output)
df$period = as.factor(df$period)
df$laser = as.factor(df$laser)
df$value = as.numeric(df$value)

Outliers

In [ ]:
%%R
df %>%
  group_by(group,test,output,laser) %>%
  identify_outliers(value)

# A tibble: 35 x 9
   laser group test  output mouse  value period  is.outlier is.extreme
   <fct> <fct> <chr> <fct>  <chr>  <dbl> <fct>   <lgl>      <lgl>     
 1 OFF   ChR2  TST   DRN    LHA95   57.2 period1 TRUE       TRUE      
 2 OFF   ChR2  TST   DRN    LHA100  37.5 period2 TRUE       FALSE     
 3 OFF   ChR2  TST   DRN    LHA101  32.2 period2 TRUE       FALSE     
 4 OFF   ChR2  TST   DRN    LHA101  44.4 period5 TRUE       FALSE     
 5 OFF   ChR2  TST   LHb    LHA95  111.  period1 TRUE       TRUE      
 6 OFF   ChR2  TST   LHb    LHA98   59.2 period1 TRUE       TRUE      
 7 OFF   ChR2  TST   LHb    LHA99   59.8 period1 TRUE       TRUE      
 8 ON    ChR2  TST   LHb    LHA93   59.7 period1 TRUE       FALSE     
 9 ON    ChR2  TST   LHb    LHA95   58.1 period2 TRUE       FALSE     
10 ON    ChR2  TST   LHb    LHA98   76.3 period3 TRUE       FALSE     
# … with 25 more rows


Normality

In [ ]:
%%R
shapiro <- df %>%
  group_by(group,output,laser) %>%
  shapiro_test(value)
print(shapiro,n=24)

ANOVA

In [ ]:
%%R
res.aov <- anova_test(data=df, dv=value, wid=mouse, between=group, within=c(output,period,laser))
get_anova_table(res.aov)

ANOVA Table (type III tests)

                      Effect DFn DFd     F     p p<.05   ges
1                      group   1   5 5.623 0.064       0.034
2                     output   2  10 3.371 0.076       0.073
3                     period   4  20 4.166 0.013     * 0.092
4                      laser   1   5 0.347 0.581       0.002
5               group:output   2  10 0.165 0.850       0.004
6               group:period   4  20 0.155 0.959       0.004
7                group:laser   1   5 7.551 0.040     * 0.050
8              output:period   8  40 0.628 0.749       0.022
9               output:laser   2  10 0.060 0.942       0.001
10              period:laser   4  20 1.374 0.278       0.039
11       group:output:period   8  40 0.728 0.666       0.025
12        group:output:laser   2  10 0.049 0.952       0.001
13        group:period:laser   4  20 0.734 0.580       0.021
14       output:period:laser   8  40 0.356 0.937       0.018
15 group:output:period:laser   8  40 0.328 0.950       

In [ ]:
%%R
two.way <- df %>%
  group_by(group,output) %>%
  anova_test(dv = value, wid = mouse, within = c(period,laser))
get_anova_table(two.way)

# A tibble: 18 x 9
   group output Effect         DFn   DFd     F     p `p<.05`   ges
   <fct> <fct>  <chr>        <dbl> <dbl> <dbl> <dbl> <chr>   <dbl>
 1 ChR2  DRN    period        4    20    0.843 0.514 ""      0.045
 2 ChR2  DRN    laser         1     5    0.957 0.373 ""      0.066
 3 ChR2  DRN    period:laser  4    20    0.82  0.527 ""      0.042
 4 ChR2  LHb    period        4    24    2.25  0.094 ""      0.086
 5 ChR2  LHb    laser         1     6    0.53  0.494 ""      0.005
 6 ChR2  LHb    period:laser  1.61  9.66 1.07  0.366 ""      0.078
 7 ChR2  VTA    period        4    32    2.10  0.104 ""      0.059
 8 ChR2  VTA    laser         1     8    7.85  0.023 "*"     0.173
 9 ChR2  VTA    period:laser  4    32    3.41  0.02  "*"     0.109
10 eYFP  DRN    period        1.34  9.37 3.02  0.109 ""      0.179
11 eYFP  DRN    laser         1     7    3.03  0.125 ""      0.011
12 eYFP  DRN    period:laser  1.6  11.2  0.618 0.522 ""      0.029
13 eYFP  LHb    period        1.72  8.61 0.

In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ laser, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
pwc

# A tibble: 6 x 8
  group output group1 group2 conf.low conf.high    p.adj p.adj.signif
  <fct> <fct>  <chr>  <chr>     <dbl>     <dbl>    <dbl> <chr>       
1 ChR2  DRN    OFF    ON       -0.329    9.33   0.0674   ns          
2 ChR2  LHb    OFF    ON       -0.589   16.7    0.0674   ns          
3 ChR2  VTA    OFF    ON        5.02    15.2    0.000163 ***         
4 eYFP  DRN    OFF    ON      -10.8      2.24   0.196    ns          
5 eYFP  LHb    OFF    ON       -5.97     3.64   0.629    ns          
6 eYFP  VTA    OFF    ON       -6.34     0.0837 0.0561   ns          


In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ period, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
print(pwc,n=60)

# A tibble: 60 x 8
   group output group1  group2  conf.low conf.high   p.adj p.adj.signif
   <fct> <fct>  <chr>   <chr>      <dbl>     <dbl>   <dbl> <chr>       
 1 ChR2  DRN    period1 period2   -10.8     11.5   1       ns          
 2 ChR2  DRN    period1 period3   -12.7      8.88  0.988   ns          
 3 ChR2  DRN    period1 period4   -14.7      7.56  0.899   ns          
 4 ChR2  DRN    period1 period5   -13.1      8.50  0.976   ns          
 5 ChR2  DRN    period2 period3   -13.4      8.87  0.98    ns          
 6 ChR2  DRN    period2 period4   -15.3      7.54  0.876   ns          
 7 ChR2  DRN    period2 period5   -13.7      8.49  0.965   ns          
 8 ChR2  DRN    period3 period4   -12.8      9.46  0.994   ns          
 9 ChR2  DRN    period3 period5   -11.2     10.4   1       ns          
10 ChR2  DRN    period4 period5    -9.84    12.4   0.998   ns          
11 ChR2  LHb    period1 period2   -30.6      8.82  0.539   ns          
12 ChR2  LHb    period1 period3   -27.9     1

# TST number of bouts FigI,K (left)


In [ ]:
%%R
df <- read_excel("fig7.xlsx", sheet = "TSTnumber")
df$group = as.factor(df$group)
df$output = as.factor(df$output)
df$period = as.factor(df$period)
df$laser = as.factor(df$laser)
df$value = as.numeric(df$value)

Outliers

In [ ]:
%%R
df %>%
  group_by(group,test,output,laser) %>%
  identify_outliers(value)

# A tibble: 2 x 9
  laser group test  output mouse  value period  is.outlier is.extreme
  <fct> <fct> <chr> <fct>  <chr>  <dbl> <fct>   <lgl>      <lgl>     
1 ON    eYFP  TST   DRN    LHA35     17 period5 TRUE       FALSE     
2 ON    eYFP  TST   LHb    LHA105    12 period5 TRUE       FALSE     


No extreme outliers

Normality

In [ ]:
%%R
shapiro <- df %>%
  group_by(group,output,laser) %>%
  shapiro_test(value)
shapiro

ANOVA

In [ ]:
%%R
res.aov <- anova_test(data=df, dv=value, wid=mouse, between=group, within=c(output,period,laser))
get_anova_table(res.aov)

ANOVA Table (type III tests)

                      Effect  DFn    DFd     F     p p<.05     ges
1                      group 1.00  15.00 0.185 0.673       0.00100
2                     output 2.00  30.00 0.622 0.544       0.01000
3                     period 2.09  31.41 1.792 0.182       0.01400
4                      laser 1.00  15.00 0.145 0.708       0.00051
5               group:output 2.00  30.00 0.630 0.539       0.01000
6               group:period 2.09  31.41 1.839 0.174       0.01500
7                group:laser 1.00  15.00 3.985 0.064       0.01400
8              output:period 8.00 120.00 0.432 0.900       0.00600
9               output:laser 2.00  30.00 0.607 0.551       0.00500
10              period:laser 4.00  60.00 0.444 0.776       0.00200
11       group:output:period 8.00 120.00 1.658 0.116       0.02100
12        group:output:laser 2.00  30.00 0.168 0.847       0.00100
13        group:period:laser 4.00  60.00 2.073 0.096       0.01000
14       output:period:laser 8.0

In [ ]:
%%R
two.way <- df %>%
  group_by(group,output) %>%
  anova_test(dv = value, wid = mouse, within = c(period,laser))
get_anova_table(two.way)

# A tibble: 18 x 9
   group output Effect         DFn   DFd     F     p `p<.05`      ges
   <fct> <fct>  <chr>        <dbl> <dbl> <dbl> <dbl> <chr>      <dbl>
 1 ChR2  DRN    period        2.72  21.8 1.79  0.183 ""      0.038   
 2 ChR2  DRN    laser         1      8   0.981 0.351 ""      0.04    
 3 ChR2  DRN    period:laser  4     32   1.49  0.228 ""      0.039   
 4 ChR2  LHb    period        4     32   2.36  0.074 ""      0.063   
 5 ChR2  LHb    laser         1      8   0.777 0.404 ""      0.031   
 6 ChR2  LHb    period:laser  4     32   0.326 0.859 ""      0.01    
 7 ChR2  VTA    period        4     32   0.581 0.678 ""      0.015   
 8 ChR2  VTA    laser         1      8   0.124 0.734 ""      0.004   
 9 ChR2  VTA    period:laser  4     32   2.34  0.076 ""      0.046   
10 eYFP  DRN    period        2.28  18.2 0.347 0.738 ""      0.017   
11 eYFP  DRN    laser         1      8   1.52  0.252 ""      0.006   
12 eYFP  DRN    period:laser  4     32   0.769 0.553 ""      0.019   
1

In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ laser, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
pwc

# A tibble: 6 x 8
  group output group1 group2 conf.low conf.high  p.adj p.adj.signif
  <fct> <fct>  <chr>  <chr>     <dbl>     <dbl>  <dbl> <chr>       
1 ChR2  DRN    OFF    ON       -2.32     0.0946 0.0704 ns          
2 ChR2  LHb    OFF    ON       -2.03     0.204  0.108  ns          
3 ChR2  VTA    OFF    ON       -1.45     0.779  0.553  ns          
4 eYFP  DRN    OFF    ON       -0.803    1.69   0.481  ns          
5 eYFP  LHb    OFF    ON       -1.43     1.18   0.849  ns          
6 eYFP  VTA    OFF    ON       -0.292    2.56   0.118  ns          


In [ ]:
%%R
pwc <- df %>%
  group_by(group,output) %>%
  tukey_hsd(value ~ period, paired = TRUE, p.adjust.method = "bonferroni") %>%
  select(-term,-null.value,-estimate) 
print(pwc,n=60)

# A tibble: 60 x 8
   group output group1  group2  conf.low conf.high p.adj p.adj.signif
   <fct> <fct>  <chr>   <chr>      <dbl>     <dbl> <dbl> <chr>       
 1 ChR2  DRN    period1 period2   -4.17      1.28  0.579 ns          
 2 ChR2  DRN    period1 period3   -3.94      1.50  0.721 ns          
 3 ChR2  DRN    period1 period4   -4.22      1.22  0.542 ns          
 4 ChR2  DRN    period1 period5   -3.67      1.78  0.869 ns          
 5 ChR2  DRN    period2 period3   -2.50      2.94  0.999 ns          
 6 ChR2  DRN    period2 period4   -2.78      2.67  1     ns          
 7 ChR2  DRN    period2 period5   -2.22      3.22  0.986 ns          
 8 ChR2  DRN    period3 period4   -3.00      2.44  0.999 ns          
 9 ChR2  DRN    period3 period5   -2.44      3.00  0.999 ns          
10 ChR2  DRN    period4 period5   -2.17      3.28  0.979 ns          
11 ChR2  LHb    period1 period2   -0.976     3.98  0.446 ns          
12 ChR2  LHb    period1 period3   -1.86      3.09  0.959 ns          
1